# Train Model

In [21]:
import sys
sys.path.append("..")
%reload_ext autoreload
%autoreload 2

In [22]:
import pandas as pd
from sklearn_pandas import cross_val_score, DataFrameMapper
from pandas import Categorical
from sklearn.ensemble import RandomForestClassifier
import mwapi

In [23]:
from ares.models import featurize, load_wp10

In [24]:
input_file = "../data/enwiki.labeling_revisions.w_features.nettrom_30k.csv.gz"
revisions = load_wp10(input_file)

# Create a pipeline

In [25]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer, PolynomialFeatures

In [26]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegressionCV
from sklearn.base import BaseEstimator, TransformerMixin

sqrt_cols = ['words',
             'headings',
             'sub_headings',
             'images',
             'categories',
             'wikilinks',

             'who_templates',
             'main_templates',
             'cite_templates',
             # infobox as a binary
             'citation_needed',
             'other_templates',

             'ref',
             'smartlists',
             # coordinates
             'coordinates']

binarized_cols = ['coordinates', 'infoboxes']

mapper = DataFrameMapper([
    (sqrt_cols, FunctionTransformer(func=np.sqrt)),
    (binarized_cols, FunctionTransformer(func=lambda x: x.astype(bool)))
])

# clf = LogisticRegressionCV(multi_class='multinomial', 
#                           random_state=1234, 
#                           penalty='l2',
#                           fit_intercept=True, 
#                           n_jobs=-1)
clf = RandomForestClassifier(bootstrap=True, oob_score=True, n_estimators=100)

In [27]:
pipe = Pipeline([
    ('mapper', mapper),
    # ('poly', PolynomialFeatures(degree=2)),
    # ('PCA', PCA(whiten=True)),
    ('clf', clf)
])

# Fit Model on Full Sample

In [28]:
categories = list(revisions['wp10'].dtype.categories)

In [29]:
fitted = pipe.fit(X=revisions.copy(), y=revisions['wp10'])

In [34]:
fitted.named_steps['clf'].oob_score_

0.6025783370342955

In [31]:
import dill

In [32]:
with open("../data/classifier-1.pkl", "wb")  as f:
    dill.dump(fitted, f)

# Evaluate Model Peformance

# TODO

1. Cross validate
2. Out of sample
3. Other